In [ ]:
# エラーが出る
import MeCab

In [ ]:
!curl -kL https://bootstrap.pypa.io/get-pip.py | python3

!apt-get update -y
!apt-get upgrade -y
# 👆について、十分な空き容量がないとエラーになる場合がある

!apt-get -yV install swig-doc
!apt-get -yV install swig-examples
!apt-get -yV install swig2.0-doc
!apt-get -yV install swig2.0-
!apt-get -yV install swig2.0 
!apt-get -yV install swig

!apt-get install mecab libmecab-dev mecab-ipadic-utf8 -y

!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git /tmp/work/
# 👆のclone先をホストOSとの共有ディレクトリにしているとエラーとなる場合がある。
# そのためtmp/workに保存している

!mkdir /var/lib/mecab/dic/mecab-ipadic-neologd

!apt-get install file -y

!/tmp/work/bin/install-mecab-ipadic-neologd -n -p /var/lib/mecab/dic/mecab-ipadic-neologd -y

!pip install mecab-python3

In [ ]:
import MeCab
import sys
import MeCab
m = MeCab.Tagger ("-d /var/lib/mecab/dic/mecab-ipadic-neologd")
print(m.parse ("すもももももももものうち"))

In [ ]:
# word2vecの準備
!pip install gensim

In [ ]:
#分かち書き作成
# 学習済みモデルを作成するため。分かち書きファイルを作成
import MeCab
import os, glob

wakati_file = './data/wakati.txt'

def wakati(text): 
        result = []
        #各行に分ける(windowsの改行コード\r\nを想定)
        lines = text.split('\r') 
        for line in lines:
            line = line.replace('\n','')

            # 形態素解析
            tagger = MeCab.Tagger("-d /var/lib/mecab/dic/mecab-ipadic-neologd")

            for chunk in tagger.parse(line).splitlines()[:-1]:
                (surface, hinshi) = chunk.split('\t')
                if hinshi.startswith('名詞') or hinshi.startswith('動詞') or hinshi.startswith('形容詞'):
    #           文章の一番最初に出てくる名詞or動詞or形容詞について処理を実行する                
                    if '*' in hinshi.split(",")[6]:
    #                     基本形が存在しない場合、表層形を返す
                        result.append(surface)
                    else :
    #                     基本形を返す
                        result.append(hinshi.split(",")[6]) 
                elif hinshi.startswith('感動詞'):                    
                    result.append(surface)         
        return result        

def main(file):
    words = []
    file_dir = os.path.abspath(file)
    try:
        bindata = open(file_dir, 'rb').read()     
        text = bindata.decode('shift_jis')        
        words = wakati(text)
    except:        
        try:
            text = bindata.decode('utf-8')
        except:
            try:
                text = bindata.decode('cp932')
            except Exception as e:
                print('error!',e)                
                exit(0)

    with open(wakati_file, 'a', encoding='utf-8') as f:
        f.write(' '.join(words))

if __name__ == '__main__':
    if os.path.exists(wakati_file):
#         すでにわかち書きが存在する場合削除
        os.remove(wakati_file)
        print('remove -> ' + wakati_file)
#   learningフォルダ直下のテキストファイルを読み込む
    files = glob.glob('./data/learning/*.txt')
    for file in files:
        if not 'wakati' in file:
            print(file)
            main(file)

In [ ]:
# word2vecでモデル作成
from gensim.models import word2vec

wakati_file = './data/wakati.txt'
model_file = './data/daily_w2v_model.model'

if __name__ == '__main__':
    file_dir = wakati_file
    w2v_data = word2vec.LineSentence(file_dir)
    model = word2vec.Word2Vec(w2v_data, size=100, window=3, hs=1, min_count=1, sg=1)
    model.save(model_file)

In [ ]:
# マルコフ辞書の作成
import MeCab
import json, os, glob

markov_file = "./data/markov_dict.json"
dic = {}

def wakati(text):
    wordlist = []
    lines = text.split('\r') 
    for line in lines:
        line = line.replace('\n','')

        # 形態素解析
        tagger = MeCab.Tagger("-d /var/lib/mecab/dic/mecab-ipadic-neologd")

        for chunk in tagger.parse(line).splitlines()[:-1]:
            (surface, hinshi) = chunk.split('\t')

            if  hinshi not  in ["BOS/EOS"]:
                if hinshi.startswith('記号'):
#                   記号を辞書に登録しない
                    pass
                elif '*' in hinshi.split(",")[6]:                    
#                   基本形が存在しない場合、表層形を返す
                    wordlist.append([surface,hinshi])
                else :
#                   基本形を返す
                    wordlist.append([hinshi.split(",")[6],hinshi])

    return wordlist

def regist_dic(wordlist):
    global dic
    w1 = ""
    w2 = ""

    # 要素が3未満の場合は、何もしない
    if len(wordlist) < 3 : return

    for w in wordlist :
        word = w[0]
        if word == "" or  word == "\r\n" or word == "\n" : continue
        # 辞書に単語を設定
        if w1 and w2 :
            set_dic(dic,w1, w2, word)
        # 文末を表す語のの場合、連鎖をクリアする
        if word == "。" or word == "?" or  word == "？" :
            w1 = ""
            w2 = ""
            continue
        # 次の前後関係を登録するために、単語をスライド
        w1, w2 = w2, word

    # 辞書を保存
    json.dump(dic, open(markov_file,"w", encoding="utf-8"))

# 辞書に単語を設定 --- (*2)
def set_dic(dic, w1, w2, w3):
    # 新しい単語の場合は、新しい辞書オブジェクトを作成
    if w1 not in dic : dic[w1] = {}
    if w2 not in dic[w1] : dic[w1][w2] = {}
    if w3 not in dic[w1][w2]: dic[w1][w2][w3] = 0
    # 単語の出現数をインクリメントする
    dic[w1][w2][w3] += 1

if __name__ == '__main__':
        if os.path.exists(markov_file):
#         すでにわかち書きが存在する場合削除
            os.remove(markov_file)
            print('remove -> ' + markov_file)
        files = glob.glob('./data/learning/*.txt')
        for file in files:
            if not 'wakati' in file:
                print(file)
                try:
                    bindata = open(file, 'rb').read()
                    text = bindata.decode('shift_jis')
                except:   
                    try:
                        text = bindata.decode('utf-8')
                    except:
                        try:
                            text = bindata.decode('cp932')
                        except Exception as e:
                            print('error!',e)  
                            exit(0)

                wordlist = wakati(text)
                regist_dic(wordlist)

In [ ]:
# chatを開始
import MeCab
from gensim.models import word2vec
import json, random, os

markov_dict = "./data/markov_dict.json"
model_file = './data/daily_w2v_model.model'


def preprocessing(sentence):
    return sentence.rstrip()

def talk_markov(text): 
        # 文章整形
        if text[-1] != "." or text[-1] != "?" or text[-1] != "？" or text[-1] != "!" or text[-1] != "！":                     
            text = text[:-1]
            text += '。'

        # 形態素解析
        tagger = MeCab.Tagger("-d /var/lib/mecab/dic/mecab-ipadic-neologd")

        sentence = preprocessing(text)

        for chunk in tagger.parse(sentence).splitlines()[:-1]:
            (surface, hinshi) = chunk.split('\t')
            if hinshi.startswith('名詞') or hinshi.startswith('動詞') or hinshi.startswith('形容詞'):
#           文章の一番最初に出てくる名詞or動詞or形容詞について処理を実行する                
                if '*' in hinshi.split(",")[6]:
#                     基本形が存在しない場合、表層形を返す
                    return surface
                else :
#                     基本形を返す
                    return hinshi.split(",")[6]
            elif hinshi.startswith('感動詞'):
#           　　感動詞の場合そのまま返答
                return '@' + surface

        return '@'

def load_w2v(word):
    model = word2vec.Word2Vec.load(model_file)
    try:
        similar_words = model.wv.most_similar(positive=[word])
        return random.choice([w[0] for w in similar_words])
    except:
        return word

def word_choice(sel):
    keys = sel.keys()
    ran = random.choice(list(keys))
    return ran

def make_sentence(reply):
    if '@' in reply:
#       感動詞の場合そのまま返す
        return word.replace('@','')

    markov_dic = json.load(open(markov_dict))
    if not reply == '':
        ret = []
        try:
            top = markov_dic[reply]
            word1 = word_choice(top)
            word2 = word_choice(top[word1])
            ret.append(reply)
            ret.append(word1)
            ret.append(word2)
        except Exception as e:
#             print('error:',e)
            return 'Sorry. There is no vocabulary.'

        while True:
            word3 = word_choice(markov_dic[word1][word2])
            ret.append(word3)
            if word3 == '。':
                break
            if len(ret) >= 10:
                ret.append('。')
                break
            word1, word2 = word2, word3
        return ''.join(ret)
    else:
        return ''

if __name__ == '__main__':
    while True:
        input_message = input('you -> ')
        if input_message == 'bay' or input_message == '':
            print('bot -> ' + 'bay !')
            break
            exit(0)
        word = talk_markov(input_message)
        if not '@' == word:
            reply = load_w2v(word)
        else:
#           名詞・動詞・形容詞・感動詞以外の場合
            reply = ''
        sentence = make_sentence(reply)
        print('bot -> ' + sentence)